In [1]:
from procan_connectome.config import DATA_PATH, PLOT_PATH, RANDOM_STATE, LOGGER_LEVEL
import numpy as np 
import pandas as pd
import os  

In [2]:
LOGGER_LEVEL

10

In [2]:
df = pd.read_csv(os.path.join(DATA_PATH, 'combined_datasets.csv'))

In [155]:
# df = df.set_index('ID')
df

,fun-HeadMovement,label,fun-SumEdgeNum_Between_Module02_05,fun-SumEdgeNum_Between_Module04_07,fun-SumEdgeNum_Between_Module01_08,fun-SumEdgeNum_Between_Module04_05,fun-SumEdgeNum_Within_Module03,fun-SumEdgeNum_Within_Module02,fun-SumEdgeNum_Between_Module03_05,fun-SumEdgeNum_Between_Module02_06,...,str-CustomPc_normalized-Temporal Sup L,str-CustomPc_normalized-Temporal Sup R,str-CustomPc_normalized-Temporal Pole Sup L,str-CustomPc_normalized-Temporal Pole Sup R,str-CustomPc_normalized-Temporal Mid L,str-CustomPc_normalized-Temporal Mid R,str-CustomPc_normalized-Temporal Pole Mid L,str-CustomPc_normalized-Temporal Pole Mid R,str-CustomPc_normalized-Temporal Inf L,str-CustomPc_normalized-Temporal Inf R
ID,,,,,,,,,,,,,,,,,,,,,
CAL_004,0.045966,Stage_0,48.0,12.0,21.0,14.0,4.0,75.0,34.0,55.0,...,0.982290,0.963543,0.971817,0.964923,0.983479,0.941668,0.962406,0.940476,0.947735,0.959402
CAL_005,0.047396,Stage_1b,57.0,27.0,0.0,30.0,7.0,81.0,17.0,9.0,...,0.973570,0.959821,0.980544,0.963294,0.992945,0.959436,0.972796,0.941429,0.975446,0.916014
CAL_007,0.078991,Stage_1b,53.0,37.0,17.0,24.0,3.0,59.0,21.0,21.0,...,0.976676,0.895044,0.976801,0.970409,0.981110,0.969540,0.965412,0.943672,0.972185,0.957062
CAL_008,0.077669,Stage_1a,37.0,13.0,27.0,24.0,4.0,76.0,26.0,26.0,...,0.989446,0.996458,0.976290,0.968382,0.965572,0.980069,0.919323,0.959067,0.948494,0.973390
CAL_009,0.082430,Stage_1b,87.0,35.0,13.0,38.0,3.0,63.0,15.0,22.0,...,0.970273,0.980159,0.966531,0.959067,0.974329,0.975714,0.929165,0.921749,0.959631,0.929818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
SB_040,0.096126,Stage_1a,52.0,18.0,19.0,28.0,6.0,73.0,8.0,59.0,...,0.953966,0.950581,0.973880,0.967878,0.975425,0.978427,0.968466,0.940408,0.971429,0.950915
SB_041,0.087158,Stage_1b,41.0,29.0,12.0,13.0,7.0,85.0,21.0,52.0,...,0.982290,0.949841,0.979753,0.962463,0.980298,0.948436,0.956694,0.931649,0.947108,0.930342
SB_042,0.064763,Stage_0,93.0,36.0,10.0,46.0,5.0,74.0,22.0,20.0,...,0.978427,0.939886,0.958648,0.979818,0.971193,0.971721,0.895924,0.946609,0.938571,0.951063


In [111]:
from sklearn.utils import resample
def _upsample_label(df: pd.DataFrame, label:str, n_samples:int)->pd.DataFrame: 
    label_subset = df[df['label'] == label]
    if len(label_subset) == n_samples:
        return label_subset
    else: 
        return resample(label_subset,
                        replace=True, 
                        n_samples=n_samples,
                        random_state=RANDOM_STATE,
            ) 

def _get_balanced_train(X_train, y_train, n_sample:int=None, single_label:str=None) -> tuple:
    df = X_train.join(y_train)
    dfs = []
    if n_sample is None: 
        n_sample = df.groupby('label').count().iloc[:,0].max()
    if single_label is not None: 
        dfs.append(df[df['label']!=single_label])
        dfs.append(_upsample_label(df, single_label, n_sample))
    else: 
        labels = df['label'].unique()
        for label in labels:
            dfs.append(_upsample_label(df, label, n_sample))
    df_upsampled = pd.concat(dfs)
    return df_upsampled.drop(columns=[y_train.name]), df_upsampled[y_train.name]

X_train = df.drop(columns=['label'])
y_train = df['label']
X_train, y_train = _get_balanced_train(X_train, y_train, n_sample=18, single_label='Transition')

    

In [115]:
len(X_train)

180

In [ ]:
X_train.join(y_train, how='inner')

In [117]:
X_train.iloc[0]

fun-HeadMovement                                0.045966
fun-SumEdgeNum_Between_Module02_05             48.000000
fun-SumEdgeNum_Between_Module04_07             12.000000
fun-SumEdgeNum_Between_Module01_08             21.000000
fun-SumEdgeNum_Between_Module04_05             14.000000
                                                 ...    
str-CustomPc_normalized-Temporal Mid R          0.941668
str-CustomPc_normalized-Temporal Pole Mid L     0.962406
str-CustomPc_normalized-Temporal Pole Mid R     0.940476
str-CustomPc_normalized-Temporal Inf L          0.947735
str-CustomPc_normalized-Temporal Inf R          0.959402
Name: CAL_004, Length: 1359, dtype: float64

In [120]:
y_train.iloc[0]

'Stage_0'

In [119]:
df.loc['CAL_004']['label']

'Stage_0'

In [130]:
y_train.groupby(y_train).count().min()

18

In [126]:
y_train['label'].value_counts()

KeyError: 'label'

In [108]:
X_train.join(y_train).groupby('label').count()

,fun-HeadMovement,fun-SumEdgeNum_Between_Module02_05,fun-SumEdgeNum_Between_Module04_07,fun-SumEdgeNum_Between_Module01_08,fun-SumEdgeNum_Between_Module04_05,fun-SumEdgeNum_Within_Module03,fun-SumEdgeNum_Within_Module02,fun-SumEdgeNum_Between_Module03_05,fun-SumEdgeNum_Between_Module02_06,fun-SumEdgeNum_Within_Module06,...,str-CustomPc_normalized-Temporal Sup L,str-CustomPc_normalized-Temporal Sup R,str-CustomPc_normalized-Temporal Pole Sup L,str-CustomPc_normalized-Temporal Pole Sup R,str-CustomPc_normalized-Temporal Mid L,str-CustomPc_normalized-Temporal Mid R,str-CustomPc_normalized-Temporal Pole Mid L,str-CustomPc_normalized-Temporal Pole Mid R,str-CustomPc_normalized-Temporal Inf L,str-CustomPc_normalized-Temporal Inf R
label,,,,,,,,,,,,,,,,,,,,,
HC,33,33,33,33,33,33,33,33,33,33,...,33,33,33,33,33,33,33,33,33,33
Stage_0,31,31,31,31,31,31,31,31,31,31,...,31,31,31,31,31,31,31,31,31,31
Stage_1a,37,37,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
Stage_1b,61,61,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
Transition,56,56,56,56,56,56,56,56,56,56,...,56,56,56,56,56,56,56,56,56,56


In [86]:
X_train = df.drop(columns=['label'])
y_train = df['label']
y_train.name

'label'

In [48]:
upsample

,ID,fun-HeadMovement,fun-SumEdgeNum_Between_Module02_05,fun-SumEdgeNum_Between_Module04_07,fun-SumEdgeNum_Between_Module01_08,fun-SumEdgeNum_Between_Module04_05,fun-SumEdgeNum_Within_Module03,fun-SumEdgeNum_Within_Module02,fun-SumEdgeNum_Between_Module03_05,fun-SumEdgeNum_Between_Module02_06,...,str-CustomPc_normalized-Temporal Sup R,str-CustomPc_normalized-Temporal Pole Sup L,str-CustomPc_normalized-Temporal Pole Sup R,str-CustomPc_normalized-Temporal Mid L,str-CustomPc_normalized-Temporal Mid R,str-CustomPc_normalized-Temporal Pole Mid L,str-CustomPc_normalized-Temporal Pole Mid R,str-CustomPc_normalized-Temporal Inf L,str-CustomPc_normalized-Temporal Inf R,label
102,CAL_157,0.040647,34.0,35.0,9.0,14.0,5.0,47.0,26.0,45.0,...,0.980722,0.978427,0.983771,0.963989,0.958678,0.959821,0.949743,0.945131,0.953955,Stage_1b
92,CAL_141,0.030065,44.0,30.0,12.0,37.0,5.0,89.0,21.0,18.0,...,0.987320,0.992653,0.976457,0.970851,0.990476,0.949233,0.916014,0.952909,0.982857,HC
14,CAL_023,0.072543,35.0,33.0,19.0,30.0,6.0,59.0,24.0,16.0,...,0.949743,0.980298,0.981458,0.980000,0.975446,0.984375,0.955017,0.977679,0.960000,Transition
106,CAL_163,0.071213,79.0,33.0,46.0,34.0,4.0,68.0,16.0,46.0,...,0.917396,0.973795,0.950437,0.974026,0.942411,0.845714,0.937618,0.970321,0.913334,Stage_1b
71,CAL_105,0.064630,42.0,20.0,14.0,20.0,6.0,71.0,26.0,57.0,...,0.951814,0.979746,0.982699,0.977826,0.912698,0.933036,0.929506,0.960000,0.954286,Stage_1b
20,CAL_031,0.245490,30.0,13.0,37.0,5.0,8.0,70.0,23.0,38.0,...,0.971796,0.964571,0.956049,0.975994,0.977679,0.958307,0.933319,0.948531,0.967930,Stage_1b
102,CAL_157,0.040647,34.0,35.0,9.0,14.0,5.0,47.0,26.0,45.0,...,0.980722,0.978427,0.983771,0.963989,0.958678,0.959821,0.949743,0.945131,0.953955,Stage_1b
121,CAL_181,0.038448,56.0,39.0,8.0,40.0,4.0,79.0,16.0,26.0,...,0.989298,0.952906,0.942857,0.989912,0.987055,0.933297,0.950857,0.962406,0.961829,Stage_1b
74,CAL_109,0.035643,47.0,33.0,41.0,13.0,4.0,67.0,24.0,29.0,...,0.963543,0.962406,0.966490,0.986150,0.942274,0.959402,0.936602,0.951603,0.927582,Stage_1b


,ID,fun-HeadMovement,fun-SumEdgeNum_Between_Module02_05,fun-SumEdgeNum_Between_Module04_07,fun-SumEdgeNum_Between_Module01_08,fun-SumEdgeNum_Between_Module04_05,fun-SumEdgeNum_Within_Module03,fun-SumEdgeNum_Within_Module02,fun-SumEdgeNum_Between_Module03_05,fun-SumEdgeNum_Between_Module02_06,...,str-CustomPc_normalized-Temporal Sup L,str-CustomPc_normalized-Temporal Sup R,str-CustomPc_normalized-Temporal Pole Sup L,str-CustomPc_normalized-Temporal Pole Sup R,str-CustomPc_normalized-Temporal Mid L,str-CustomPc_normalized-Temporal Mid R,str-CustomPc_normalized-Temporal Pole Mid L,str-CustomPc_normalized-Temporal Pole Mid R,str-CustomPc_normalized-Temporal Inf L,str-CustomPc_normalized-Temporal Inf R
label,,,,,,,,,,,,,,,,,,,,,
HC,33,33,33,33,33,33,33,33,33,33,...,33,33,33,33,33,33,33,33,33,33
Stage_0,31,31,31,31,31,31,31,31,31,31,...,31,31,31,31,31,31,31,31,31,31
Stage_1a,37,37,37,37,37,37,37,37,37,37,...,37,37,37,37,37,37,37,37,37,37
Stage_1b,61,61,61,61,61,61,61,61,61,61,...,61,61,61,61,61,61,61,61,61,61
Transition,18,18,18,18,18,18,18,18,18,18,...,18,18,18,18,18,18,18,18,18,18


In [71]:
df[df['label']=='Transition']

,ID,fun-HeadMovement,label,fun-SumEdgeNum_Between_Module02_05,fun-SumEdgeNum_Between_Module04_07,fun-SumEdgeNum_Between_Module01_08,fun-SumEdgeNum_Between_Module04_05,fun-SumEdgeNum_Within_Module03,fun-SumEdgeNum_Within_Module02,fun-SumEdgeNum_Between_Module03_05,...,str-CustomPc_normalized-Temporal Sup L,str-CustomPc_normalized-Temporal Sup R,str-CustomPc_normalized-Temporal Pole Sup L,str-CustomPc_normalized-Temporal Pole Sup R,str-CustomPc_normalized-Temporal Mid L,str-CustomPc_normalized-Temporal Mid R,str-CustomPc_normalized-Temporal Pole Mid L,str-CustomPc_normalized-Temporal Pole Mid R,str-CustomPc_normalized-Temporal Inf L,str-CustomPc_normalized-Temporal Inf R
8,CAL_014,0.158240,Transition,61.0,33.0,47.0,28.0,7.0,61.0,23.0,...,0.950437,0.958171,0.965762,0.963400,0.966490,0.959283,0.921521,0.892857,0.965714,0.961199
14,CAL_023,0.072543,Transition,35.0,33.0,19.0,30.0,6.0,59.0,24.0,...,0.989298,0.949743,0.980298,0.981458,0.980000,0.975446,0.984375,0.955017,0.977679,0.960000
21,CAL_032,0.031973,Transition,76.0,28.0,12.0,19.0,4.0,60.0,24.0,...,0.975238,0.954921,0.979753,0.970522,0.964581,0.940035,0.913334,0.932778,0.953174,0.932224
23,CAL_036,0.081042,Transition,29.0,18.0,0.0,13.0,6.0,63.0,17.0,...,0.936602,0.968123,0.965015,0.987654,0.968039,0.963400,0.907372,0.956995,0.947522,0.969143
31,CAL_048,0.089061,Transition,67.0,25.0,5.0,36.0,7.0,81.0,25.0,...,0.980114,0.988571,0.974409,0.956684,0.986747,0.963400,0.961301,0.900794,0.959202,0.962120
37,CAL_058,0.073071,Transition,115.0,25.0,37.0,39.0,9.0,90.0,15.0,...,0.975994,0.965486,0.970409,0.933036,0.988631,0.953040,0.963282,0.938213,0.958142,0.963400
44,CAL_067,0.048687,Transition,60.0,25.0,6.0,36.0,5.0,76.0,24.0,...,0.939498,0.963543,0.967598,0.981145,0.969226,0.977177,0.966634,0.949841,0.971817,0.962054
69,CAL_103,0.051718,Transition,84.0,30.0,32.0,21.0,6.0,81.0,26.0,...,0.960000,0.979592,0.960000,0.981401,0.974409,0.953102,0.941799,0.962799,0.967936,0.970018
72,CAL_106,0.041634,Transition,48.0,13.0,18.0,19.0,7.0,77.0,16.0,...,0.984127,0.976457,0.980192,0.966490,0.962054,0.979911,0.935065,0.944254,0.959436,0.946609


In [145]:
import datetime
import time 
start = datetime.datetime.now()
time.sleep(2.0)
end=datetime.datetime.now()
print(end-start)

0:00:02.002241


In [152]:
start = datetime.datetime.now()
start.strftime('%Y_%m_%d_%H_%M')

'2021_06_22_23_12'

In [154]:
datetime.datetime.now().strftime('%Y_%m_%d_%H_%M')

'2021_06_22_23_14'